## **Train the model on the Google AI Platform**

In [ ]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()
## Mount on google drive
from google.colab import drive
drive.mount('/content/drive/')
# # Import and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

### <font color=red> **！！Note: the runtime-version setting (supported tensorflow version in AI platform) should be consistency with the Tensorflow version in the colab, and we use Tensorflow 2.2.0 here!**

In [ ]:
# !pip install tensorflow==2.2.0


In [ ]:
import os
os.chdir("/content/drive/My Drive/Earth-Engine-with-Deep-Learning/trainer")
import tensorflow as tf
print(tf.__version__)
# Folium setup.
import folium
print(folium.__version__)

In [ ]:
Package_Path = 'ai_platform_train'

# !ls -l
# !mkdir {Package_Path}
# !touch {Package_Path}/__init__.py
# !ls -l {Package_Path}


### **Prepare the configuration file.**

In [ ]:
%%writefile {Package_Path}/config.py

import tensorflow as tf

# INSERT YOUR PROJECT HERE!
Project = 'my-project-20200813'

# INSERT YOUR BUCKET HERE!
Bucket = 'earth-engine-bucket-1'

# Specify names of output locations in Cloud Storage.
Folder = 'ai_platform_train'
Job_Dir = 'gs://' + Bucket + '/' + Folder

Model_Dir = Job_Dir + '/model'
Logs_Dir = Job_Dir + '/logs'

# Put the EEified model next to the trained model directory.
EEified_Dir = Job_Dir + '/eeified'

# Pre-computed training data.
Train_Data_Folder = 'NLCD_Impervious_Data'

# output bands
Bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
Targets = ['impervious']
Features = Bands + Targets

# Specify the size and shape of patches expected by the model.
Kernel_Size = [256, 256]
Columns = [
  tf.io.FixedLenFeature(shape=Kernel_Size, dtype=tf.float32) for k in Features
]
Features_Dict = dict(zip(Features, Columns))

# Sizes of the training datasets.
Train_Size = 1000

# Specify model training parameters.
Batch_Size = 16
Epochs = 50
Buffer_Size = 2000
Optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9)
Loss = 'MeanSquaredError'
Metrics = ['RootMeanSquaredError']

Overwriting ai_platform_train/config.py


### Check the configuration file.

In [ ]:
# !cat {PACKAGE_PATH}/config.py
from ai_platform_train import config
print('\n', config.Job_Dir)



 gs://earth-engine-bucket-1/ai_platform_train


### Data loader

In [ ]:
## Data loader
%%writefile {Package_Path}/dataLoader.py

from . import config
import tensorflow as tf

# Dataset loading functions
def parse_tfrecord(example_proto):
  return tf.io.parse_single_example(example_proto, config.Features_Dict)

def to_tuple(inputs):
  inputsList = [inputs.get(key) for key in config.Features]
  stacked = tf.stack(inputsList, axis=0)
  stacked = tf.transpose(stacked, [1, 2, 0])
  return stacked[:,:,:len(config.Bands)], stacked[:,:,len(config.Bands):]

def get_dataset(pattern):
	glob = tf.io.gfile.glob(pattern)
	dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
	dataset = dataset.map(parse_tfrecord)
	dataset = dataset.map(to_tuple)
	return dataset

def get_training_dataset():
	glob = 'gs://' + config.Bucket + '/' + config.Train_Data_Folder + '/' + '*'
	dataset = get_dataset(glob)
	dataset = dataset.shuffle(config.Buffer_Size).batch(config.Batch_Size).repeat()
	return dataset

# def get_eval_dataset():
# 	glob = 'gs://' + config.DATA_BUCKET + '/' + config.FOLDER + '/' + config.EVAL_BASE + '*'
# 	dataset = get_dataset(glob)
# 	dataset = dataset.batch(1).repeat()
# 	return dataset


Overwriting ai_platform_train/dataLoader.py


## Model 

In [ ]:
## model building
%%writefile {Package_Path}/model.py

import tensorflow as tf

############## U-Net
###  Define the downsample function
##   Conv2D+BN+ReLU
def downsample(filters, size, apply_batchnorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = tf.keras.Sequential()
    result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))
    if apply_batchnorm:
        result.add(tf.keras.layers.BatchNormalization())
#     result.add(tf.keras.layers.LeakyReLU())
    result.add(tf.keras.layers.ReLU())
    return result

### Define the upsample function
##  TransposeConv2D+BN+ReLU
def upsample(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = tf.keras.Sequential()
    result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))
    result.add(tf.keras.layers.BatchNormalization())
    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.5))
    result.add(tf.keras.layers.ReLU())
    return result

## Simple U-Net
def UNet(input_shape, nclasses=2):
    ## encoder of the U-Net
    (img_height, img_width, img_channel) = input_shape
    down_stack = [
        downsample(12, 3), # outp: (bs, img_height/2, img_width/2, 32)
        downsample(24, 3), # (bs, img_height/4, img_width/4, 64)
        downsample(48, 3), # (bs, img_height/8, img_width/8, 128)
        downsample(96, 3), # (bs, img_height/16, img_width/16, 256)
        downsample(96, 3), # (bs, img_height/32, img_width/32, 512)
        # downsample(96, 3), # (bs, img_height/64, img_width/64, 512)
        # downsample(96, 3), # (bs, img_height/128, img_width/128, 512)
    ]

    ## decoder of the U-Net
    up_stack = [
        # upsample(96, 3), # outp: (bs, img_height/64, img_width/64, 1024)
        # upsample(96, 3), # (bs, img_height/32, img_width/32, 1024)
        upsample(96, 3), # (bs, img_height/16, img_width/16, 1024)
        upsample(48, 3), # (bs, img_height/8, img_width/8, 512)
        upsample(24, 3), # (bs, img_height/4, img_width/4, 256)
        upsample(12, 3), # (bs, img_height/2, img_width/2, 128)
    ]

    initializer = tf.random_normal_initializer(0., 0.02)
    
    # define the input and output tensors
    inputs = tf.keras.layers.Input(shape=[img_height, img_width, img_channel])
    if nclasses == 2:
        last = tf.keras.layers.Conv2DTranspose(1, 3,
                            strides=2,
                            padding='same',
                            kernel_initializer=initializer,
                            activation= 'sigmoid')  ## 
    else:
        last = tf.keras.layers.Conv2DTranspose(nclasses, 3,
                            strides=2,
                            padding='same',
                            kernel_initializer=initializer,
                            activation= 'softmax')  ##
    concat = tf.keras.layers.Concatenate()    
    x = inputs
    # Downsampling through the model
    skips = []   # reserve the output of medium output of the encoder network 
    for down in down_stack:
        x = down(x)
        skips.append(x)
    skips = reversed(skips[:-1])  #  
    # Upsampling and establishing the skip connections
    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = concat([x, skip])
    x = last(x)
    return tf.keras.Model(inputs=inputs, outputs=x)


Overwriting ai_platform_train/model.py


## Check the dataloader functions and the built model

In [ ]:
from ai_platform_train import model
from ai_platform_train import dataLoader

trainData = dataLoader.get_training_dataset()
# print(iter(trainData.take(1)).next())

model = model.UNet(input_shape=(256, 256, 6), nclasses=2)
# print(model.summary())


## Training task on AI Platform

In [ ]:
%%writefile {Package_Path}/trainingTask.py

from . import config
from . import model
from . import dataLoader
import tensorflow as tf

if __name__ == '__main__':

    training = dataLoader.get_training_dataset()

    model = model.UNet(input_shape=(256, 256, 6), nclasses=2)

    model.compile(
		optimizer=tf.keras.optimizers.get(config.Optimizer),
		loss=tf.keras.losses.get(config.Loss),
		metrics=[tf.keras.metrics.get(metric) for metric in config.Metrics])

    model.fit(
        x=training,
        epochs=config.Epochs, 
        steps_per_epoch=10,
        callbacks=[tf.keras.callbacks.TensorBoard(config.Logs_Dir)]
        )

    model.save(config.Model_Dir, save_format='tf')


Overwriting ai_platform_train/trainingTask.py


## Submit the package to AI Platform for training

In [ ]:
import time

Job_Name = 'impervious_unet_training_job_' + time.strftime("%Y%m%d%H%M")
Train_Package_Path = 'ai_platform_train'
Main_Trainer_Module = 'ai_platform_train.trainingTask'
Region = 'asia-east1'


In [ ]:
!gcloud ai-platform jobs submit training {Job_Name} \
    --job-dir {config.Job_Dir} \
    --package-path {Train_Package_Path} \
    --module-name {Main_Trainer_Module} \
    --region {Region} \
    --project {config.Project} \
    --runtime-version 2.2 \
    --python-version 3.7 \
    --scale-tier basic-gpu


In [ ]:
desc = !gcloud ai-platform jobs describe {Job_Name} --project {config.Project}
state = desc.grep('state:')[0].split(':')[1].strip()
print(state)


## **Inspect the trained model and prepare the model for making predictions in *Earth* Engine**


In [ ]:
%load_ext tensorboard
%tensorboard --logdir {config.Logs_Dir}
